<center><h1> The Annotated Tnn </h1></center>


<center>
<p><a href="https://openreview.net/pdf?id=IxmWsm4xrua">Toeplitz Neural Network for Sequence Modeling</a></p>
</center>

<img src="images/network.png" width="100%"/>

*博客由[Doreamonzzz](https://github.com/Doraemonzzz)撰写。*


Toeplitz Neural Network(TNN)是一种全新的网络结构，以一种完全不同的方式进行序列建模，并且可以在单向/双向语言模型，图像分类任务上和Transformer性能相近，并且在长序列建模[LRA]()任务上取得和[S4]()相当的性能。这篇博客的主要目的就是以[The Annotated Transformer]()和[The Annotated S4]()风格介绍TNN，在阅读完这篇博客后，您将得到如下收获：
1. 了解TNN的动机和设计理念；
2. 掌握TNN各个部件的实现；
3. 学习如何将TNN应用在$n$维序列建模任务；
4. 了解TNN的优缺点；
5. 了解TNN和S4, RWKV等方法的联系；

总而言之，在阅读完本博客之后，您将成为TNN的专家，并且可以将TNN应用到您的项目中，让我们开始吧。


# 预备知识

## Token and channel mixing
让我们首先从Transformer开始。Transformer作为一个网络结构已经席卷了各个领域，其核心部分主要可以由如下两个计算公式描述：
$$
\begin{aligned}
\mathbf X_1 &=\mathrm{Norm}(\mathbf X + \mathrm{MHA}(\mathbf X)),\\
\mathbf O &= \mathrm{Norm}(\mathbf X_1 + \mathrm{FFN}(\mathbf X_1)).
\end{aligned}
$$
其中$\mathbf X \in \mathbb R^{n\times d}$是输入（也可以称为token matrix，其中矩阵的每一行为一个token的向量表示），$n$是序列长度，$d$是特征维度。

既然现在有两个主要模块——$\mathrm {MHA}$和$\mathrm {FFN}$，那么他们的作用是否有所不同呢？在[Metaformer]()一文中，研究者指出，$\mathrm {MHA}$的主要作用是Token mixing，而$\mathrm {FFN}$的主要作用是Channel mixing。

这是什么意思呢？我们可以从矩阵乘法的角度清晰的理解这点：给定输入（token matrix）$\mathbf X \in \mathbb R^{n\times d}$，考虑矩阵乘法$\mathbf A \mathbf X$和$\mathbf X \mathbf B$，那么：
- $\mathbf A \mathbf X$表示矩阵$\mathbf X$行的线性组合，而每一行表示一个token，即token的线性组合，所以称为token mixing；
- $\mathbf X  \mathbf B$表示矩阵$\mathbf X$列的线性组合，而每一列表示一个channel，即channel的线性组合，所以称为channel mixing；

在Transformer中，矩阵$\mathbf A$即为$\mathrm{Softmax}(\mathbf Q \mathbf K^{\top} /\sqrt{d})$，矩阵$B$即为$\mathrm {FFN}$中的全连接层。

大多数对Transformer的改进都是集中在token mixing:$\mathbf A \mathbf X$的计算上，以各种各样的方式降低其运算复杂度，TNN也是使用了类似的思路，最核心的一点就是利用了相对位置编码，或者说，Toeplitz矩阵。

## 相对位置编码

位置编码是Transformer中的重要组成部分，一开始广为使用的是[绝对位置编码(APE)]()，这种编码的方式可以用如下计算方式概括：
$$
\mathbf x_i =\mathbf w_i + \mathbf p_i.
$$
其中$\mathbf w_i$表示第$i$个词的word embedding，$\mathbf p_i$表示第$i$个位置的position embedding。

后来，有研究人员发现，在序列建模中，词的相对位置信息，可能比词的绝位置信息更加重要。
> 例如"我年纪比你大"的语意和"我年纪比你大"完全不同，但是这两句话只是交换了"你"和"我"的位置。
>

于是研究人员开始将相对位置编码引入，相对位置编码的使用和绝对位置编码有所不同，其作用在Attention计算的位置：
$$
\mathbf s_{ij} = \mathbf q_i^{\top} \mathbf k_j/\sqrt{d} + t_{i-j}.
$$
如果写成矩阵的形式则更加直观：
$$
\begin{aligned}
\mathbf S & = \mathbf Q \mathbf K^{\top} / \sqrt {d} + \mathbf T,\\
\mathbf T & =\left[\begin{array}{cccc}
t_0 & t_{-1} & \cdots  & t_{-n+1} \\
t_1 & t_0  &  &  \vdots \\
\vdots &   &   t_0 & t_{-1} \\
t_{n-1} & \ldots  & t_1 & t_0
\end{array}\right] \in \mathbb R^{n\times n}.
\end{aligned}
$$

这里，矩阵$\mathbf T$有一个数学名称——[Toeplitz矩阵]()，不难看出该矩阵有$2n-1$个独立元素。

# TNN的动机

有了之前的准备工作，可以引入我们工作的两个动机：
1. 既然相对位置信息如此重要，那么有没有可能只依赖于相对位置信息（Toeplitz matrix）进行token mixing呢？
   1. 直观上来说，就是将Attention Matrix替换为Toeplitz matrix。
2. 假设(1)成立，那么我们需要进行的主要操作是$\mathbf T \mathbf X$，既然矩阵$\mathbf T$是一个特殊结构的矩阵，那么有没有可能加速运算呢？

我们对两个问题都进行了肯定的答复：
1. 完全可以只依赖于相对位置信息进行token mixing；
2. 由于矩阵的特殊性，可以将运算复杂度由$O(n^2 d)$降低为$O(nd\log n)$；

可以看到，我们的动机极其简单和优雅，最核心的思路就是将$\mathrm{Softmax}(\mathbf Q \mathbf K^{\top} / \sqrt {d}) $替换为$\mathbf T$，但是，这种简单的替换就可以拥有比各种花哨更改更好的性能，这就更加验证了相对位置信息在序列建模中的重要性。


<!-- 因此，在后续的讨论快速矩阵乘法时，我们指的是及$\mathbf T\mathbf x$，其中$\mathbf T\in \mathbb R^{n\times n}, \mathbf x \in \mathbb R^{n\times 1}$。 -->

# TNN的实现


## 准备工作

接下来的问题就是如何实现TNN，在此之前，我们对之前的公式做一定的调整。

在之前的讨论中，我们提到了$\mathbf T \mathbf X$可以高效实现，其中$\mathbf T\in \mathbb R^{n\times n}, \mathbf X \in \mathbb R^{n\times d}$，这种情况相当于每个channel共享同一个Toeplitz matrix，但是注意到我们可以让不同的channel使用不同的Toeplitz matrix，我们经验上发现，这样一定程度上可以增大模型的表达性，所以在TNN中，**每个channel**使用了不同的Toeplitz matrix。注意到形状为$n\times n$的Toeplitz matrix实际上只有$2n-1$个独立元素，为了方便后续讨论，我们定义如下映射：$f: \mathbb R^{(2n-1)\times 1} \to \mathbb R^{n\times n}$：
$$
f(\mathbf t)=f(t_{-n+1},\ldots, t_{n-1}) =\left[\begin{array}{cccc}
t_0 & t_{-1} & \cdots  & t_{-n+1} \\
t_1 & t_0  &  &  \vdots \\
\vdots &   &   t_0 & t_{-1} \\
t_{n-1} & \ldots  & t_1 & t_0
\end{array}\right] \in \mathbb R^{n\times n}.
$$
该映射的作用是将维度为$(2n-1)\times 1$的向量填充为$n\times n$的Toeplitz matrix。

结合之前的记号，我们定义为Tno算子(Toeplitz neural operator)为：
$$
\mathrm{Tno}: \mathbb R^{(2n-1)\times d}\times \mathbb R^{n\times d} \to \mathbb R^{n\times d},\\
\mathbf O= \mathrm{Tno}(\mathbf T, \mathbf X), \\
\mathbf O[:, i]= f(\mathbf T[:, i]) \mathbf X[:, i].
$$

备注：这里的记号$\mathbf T\in \mathbb R^{(2n-1)\times d}$和一开始含义有所不同，注意不要搞混。


<!-- ，写成计算公式即为：
$$
\mathbf O[:, i]= \mathbf T[:, i] \mathbf X[:, i], \\
\mathbf O[:, i]\in \mathbb R^{n\times 1}, \mathbf T[:, i]\in \mathbb R^{n\times n},  \mathbf X[:, i]\in \mathbb R^{n\times 1}.
$$ -->

<!-- 备注：这里记号有点太严谨，$\mathbf T[:, i]$的形状应该是$(2n-1) \times 1$，上述符号是指 -->

## Tno的实现

### Naive实现

最朴素的实现自然是利用定义进行实现，例如如下代码中，我们使用4重循环，外面两重循环遍历batch, channel维度，第三重循环遍历输出位置，最后一重循环遍历求和项，注意到我们的$\mathbf T[:, i]$输入形式为$t_{-n+1}, ... , t_{-1}, t_0, t_1, ... , t_{n - 1} $，第三重循环遍历到$i$时，涉及的$t$为$t_{i}, t_{i-1},\ldots, t_{i-n+1}$，而$n - 1 + i$是$t_{i}$在$\mathbf T[:, i]$的实际索引：

In [57]:
import torch

def tno_naive(x, t):
    # x: (b, n, d)
    # t: (2n - 1, d), t_(-(n - 1)), ... , t_(-1), t_0, t_1, ... , t_(n - 1) 
    b, n, d = x.shape
    o = torch.zeros_like(x).to(x)
    for b_ in range(b):
        for d_ in range(d):
            for i in range(n):
                for j in range(n):
                    print(n - 1 - i + j, t[n - 1 + i - j][d_])
                    o[b_][i][d_] += t[n - 1 + i - j][d_] * x[b_][j][d_]

    return o

这种实现显然太低效，但是至少我们有了一个正确的版本，这对我们后续改进算法也是有帮助的，不难看出这样计算的时间复杂度为$O(n^2d)$，空间复杂度为$O(nd)$（忽略batch维度）。                             

### Matrix production实现
第二种实现是并行版本，其思路就是先构造Toeplitz matrix，然后利用矩阵乘法进行计算。最主要的部分是将映射$f$实现出来，代码基于[此处](https://stackoverflow.com/questions/69809789/is-there-any-way-to-create-a-tensor-with-a-specific-pattern-in-pytorch)，主要思路是先将输入改写为$t_0, t_{-1}, ... , t_{1-n}, t_{n - 1}, ... , t_1$，然后构造index $0, 1, \ldots,n -1, -(n - 1), ..., -1$，将输入映射到Toeplitz matrix，最后得到Toeplitz matrix进行矩阵乘法：

In [62]:
def tno_matrix(x, t):
    # x: (b, n, d)
    # t: (2n - 1, d), t_(-(n - 1)), ... , t_(-1), t_0, t_1, ... , t_(n - 1) 
    n = x.shape[1]
    t = t.unsqueeze(0)
    # c: t_0, t_1, ... , t_(n - 1)
    c = t[:, n - 1:]
    # r: t_0, t_(-1), ... , t_(-(n - 1))
    r = t[:, :n].flip(1)
    # vals: [t_0, t_(-1), ... , t_(-(n - 1)), t_(n - 1), ... , t_1]
    vals = torch.cat([r, c[:, 1:].flip(1)], dim=-2)
    i, j = torch.ones(n, n).nonzero().T
    t_matrix = vals[:, j - i].reshape(n, n, -1)
    o = torch.einsum("n m d, b m d -> b n d", t_matrix, x)

    return o

这种实现的好处是可以利用矩阵乘法，尽管复杂度依然为$O(n^2d)$，但实际效率会快很多；但是由于要构造Toeplitz matrix，所以空间复杂度为$O(n^2d)$，并且这部分还是一个很大的IO开销，所以实际中的速度并不会很快。

### FFT实现
有了之前的铺垫，可以看出前两种方法无论是时间复杂度和空间复杂度相比Attention并没有什么优势，那么有没有办法解决这点呢？回答是肯定的，这就需要[FFT]()这把利刃。后续的讨论涉及到一些数学知识，这里先高度概括一下思路：
1. 给出Circulant matrix的快速矩阵乘法算法；
2. 证明Toeplitz marix和Circulant matrix的关系；

### Circulant matrix

#### 定义
矩阵$\mathbf C\in \mathbb R^{n\times n}$是一个Circulant matrix当且仅当$\mathbf C_{ij}= c_{(i-j + n )\bmod n}$ ,即：
$$
\mathbf C=\left[\begin{array}{cccccc}
c_0 & c_{n-1} &c_{n-2} & \cdots & \cdots & c_{1} \\
c_1 & c_0 & c_{n-1} & \ddots & & \vdots \\
c_2 & c_1 & \ddots & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & c_{n-1} & c_{n-2} \\
\vdots & & \ddots & c_1 & c_0 & c_{n-1} \\
c_{n-1} & \ldots & \ldots & c_2 & c_1 & c_0
\end{array}\right] \in \mathbb R^{n\times n}.
$$
关于Circulant matrix，有如下重要性质：

Circulant matrix $\mathbf C\in \mathbb R^{n\times n}$正交相似于对角阵$\mathbf \Lambda$，特别地，相似矩阵$\mathbf F$是$n\times n$ DFT矩阵:
$$
\mathbf C = \mathbf F^{\top} \Lambda \mathbf F, \\
\Lambda = \mathrm{diag}\{\mathbf F[c_0,c_1,\ldots, c_{n-1}]^\top\} \in \mathbb R^{n\times n}, 
{\mathbf F}_{st}= \exp\left(\frac{2\pi st i}{n}\right),i^2=-1.
$$

#### 快速矩阵乘法

现在考虑matrix-vector production操作$\mathbf M \mathbf x, \mathbf M\in \mathbb R^{n\times n}, \mathbf x\in \mathbb R^{n\times 1}$，那么：
- 如果$\mathbf M$为一般的矩阵，那么该计算的时间复杂度为$O(n^2)$;
- 如果$\mathbf M$为DFT矩阵，那么该计算的时间复杂度为$O(n \log n)$;


基于上述事实，考虑$\mathbf M=\mathbf C$为Circulant matrix的情形，那么：
$$
\mathbf C \mathbf x = \mathbf F^{\top} \Lambda \mathbf F \mathbf x.
$$
该计算可以分解为几个步骤：
- $\mathbf x_{\mathrm{fft}}=\mathbf{Fx}$；
- $\mathbf c_{\mathrm{fft}}=\mathbf F[c_0,c_1,\ldots, c_{n-1}]^\top$；
- $\mathbf o_{\mathrm{fft}}=\mathbf x_{\mathrm{fft}}\odot \mathbf c_{\mathrm{fft}}$；
- $\mathbf o= \mathbf F^{\top} \mathbf o_{\mathrm{fft}}$；

其中$\odot$表示element-wise production，可以看出，算法的总时间复杂度为$O(n\log n)$，所以Circulant matrix对应的矩阵乘法是高效的，那么下一个问题就是建立Toeplitz matrix和Circulant matrix的关系。

#### 实现
有了之前的说明，不难利用`fft`实现上述计算：

### Toeplitz matrix
#### 定义
矩阵$\mathbf T\in \mathbb R^{n\times n}$是一个Toeplitz matrix当且仅当$\mathbf T_{ij}= t_{i-j}$，即
$$
\mathbf T=\left[\begin{array}{cccccc}
t_0 & t_{-1} &t_{-2} & \cdots & \cdots & t_{-n+1} \\
t_1 & t_0 & t_{-1} & \ddots & & \vdots \\
t_2 & t_1 & \ddots & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & t_{-1} & t_{n-2} \\
\vdots & & \ddots & t_1 & t_0 & t_{-1} \\
t_{n-1} & \ldots & \ldots & t_2 & t_1 & t_0
\end{array}\right] \in \mathbb R^{n\times n}.
$$
从形式上来看，Toeplitz matrix和Circulant matrix非常像，唯一的区别在于前者的独立元素数量为$2n-1$，后者的独立元素数量为$n$，那么一个简单的思路就是将Toeplitz matrix嵌入到一个阶数大于等于$2n-1$矩阵中，而这个矩阵本生是一个Circulant matrix，下面来看下这是如何具体操作的。

可以将Toeplitz matrix $\mathbf T\in \mathbb R^{n\times }$嵌入到Circulant matrix $\mathbf C \in \mathbb R^{2n\times 2n}$中:
$$
c_{k} =\begin{cases}
t_k , 0 \le k \le n - 1\\
t_0 , k=n\\
t_{k -2n},   n+1\le k \le 2n-1
\end{cases} ,
$$
即，
$$
\mathbf C=\left[\begin{array}{ccccc|ccccc}
t_0 & t_{-1} & \ldots & \ldots & t_{-n+1} & t_0 & t_{n-1} & \ldots & t_2 & t_1 \\
t_1 & t_0 & \ddots & & \vdots & t_{-n+1} & \ddots & \ddots & & t_2 \\
t_2 & \ddots & \ddots & \ddots & \vdots & \vdots & \ddots & & \ddots & \vdots \\
\vdots & & \ddots & t_0 & t_{-1} & t_{-2} & & \ddots & \ddots & t_{n-1} \\
t_{n-1} & \ldots & \ldots & t_1 & t_0 & t_{-1} & t_{-2} & \ldots & t_{-n+1} & t_0 \\
\hline t_0 & t_{n-1} & \ldots & \ldots & t_1 & t_0 & t_{-1} & \ldots & \ldots & t_{-n+1} \\
t_{-n+1} & \ddots & \ddots & & t_2 & t_1 & t_0 & \ddots & & \vdots \\
\vdots & \ddots & & \ddots & \vdots & t_2 & \ddots & \ddots & \ddots & \vdots \\
t_{-2} & & \ddots & \ddots & t_{n-1} & \vdots & & \ddots & t_0 & t_{-1} \\
t_{-1} & t_{-2} & \ldots & \ldots & t_0 & t_{n-1} & \ldots & \ldots & t_1 & t_0
\end{array}\right] \in \mathbb R^{2n\times 2n}.
$$

使用分块矩阵的符号，我们可以定义：
$$
\begin{gathered}
 \mathbf C = \left[\begin{array}{cc}
\mathbf C_1 & \mathbf C_2\\
\mathbf C_3 & \mathbf C_4\\
\end{array}\right] \in \mathbb R^{2n\times 2n},\mathbf C_s \in \mathbb R^{n \times n}, s=1,2,3,4,
\mathbf C_1 = \mathbf T 
\end{gathered}.
$$

有了上述准备工作，可以得到Toeplitz matrix-vector production的快速算法。

#### 快速矩阵乘法

对于向量$\mathbf x\in \mathbb R^{n}$, 定义:
$$
\mathbf x_1 = \left[\begin{array}{c}
\mathbf x\\
\mathbf 0_n
\end{array}\right] \in \mathbb R^{2n},
$$
所以，
$$
\mathbf C \mathbf x_1 =\left[\begin{array}{cc}
\mathbf C_1 & \mathbf C_2\\
\mathbf C_3 & \mathbf C_4\\
\end{array}\right]\left[\begin{array}{c}
\mathbf x\\
\mathbf 0_n
\end{array}\right]=\left[\begin{array}{c}
\mathbf C_1 \mathbf x\\
\mathbf C_3 \mathbf x
\end{array}\right]=\left[\begin{array}{c}
\mathbf T \mathbf x\\
\mathbf C_3 \mathbf x
\end{array}\right] \in \mathbb R^{2n},
$$
因此:
$$
\left[\begin{matrix}
{\mathbf I}_n &
{\mathbf 0}_{n\times n}
\end{matrix}\right]\mathbf C \mathbf x_1  =
\left[\begin{matrix}
\mathbf I_n &
\mathbf 0_{n\times n}
\end{matrix}\right]\left[\begin{array}{c}
\mathbf T \mathbf x\\
\mathbf C_3 \mathbf x
\end{array}\right]=\mathbf T \mathbf x.
$$
关于时间复杂度，注意到我们是将$n\times n$的Toeplitz matrix嵌入到一个$2n\times 2n$的Circulant matrix中，所以时间复杂度仍然为$O(n\log n)$。

#### 实现
和Circulant matrix的情形类似，可以利用`fft`实现上述计算：

### 验证实现以及速度对比

In [61]:
b = 1
n = 3
d = 1

t_zero = torch.randn(1, d)
t_pos = torch.randn(n - 1, d)
t_neg = torch.randn(n - 1, d)

t = torch.cat([t_neg, t_zero, t_pos], dim=0).cuda()
x = torch.randn(b, n, d).cuda()


print(torch.norm(tno_naive(x, t) - tno_matrix(x, t)))


torch.Size([5, 1])
2 tensor(0.3044, device='cuda:0')
3 tensor(0.5080, device='cuda:0')
4 tensor(0.7849, device='cuda:0')
1 tensor(-0.5896, device='cuda:0')
2 tensor(0.3044, device='cuda:0')
3 tensor(0.5080, device='cuda:0')
0 tensor(0.4143, device='cuda:0')
1 tensor(-0.5896, device='cuda:0')
2 tensor(0.3044, device='cuda:0')
tensor([ 0,  1,  2, -1,  0,  1, -2, -1,  0])
tensor([[[ 0.7849],
         [ 0.5080],
         [ 0.3044],
         [-0.5896],
         [ 0.4143]]], device='cuda:0')
tensor([[ 0.3044,  0.5080,  0.7849],
        [-0.5896,  0.3044,  0.5080],
        [ 0.4143, -0.5896,  0.3044]], device='cuda:0')
tensor(7.4506e-09, device='cuda:0')


### 小结
在本节中，我们从naive的算法开始，最终得到了一个基于FFT算法的高效实现。注意到Tno的计算涉及到$x,t$，$x$是输入，$t$是相对位置系数，所以下一步就是如何计算$t$。

## Rpe的实现
